In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import json
import scipy.io as sio
import cv2
import imageio

In [ ]:
%run GetLossObj.ipynb

In [ ]:
baseDir = "../save/server1/rnn/model130/"
plotTrainReconLosses = torch.load(baseDir + "plot_train_recon_losses", map_location="cpu")
plotValReconLosses = torch.load(baseDir + "plot_val_recon_losses", map_location="cpu")
with open(baseDir + "args.txt") as f:
    args = f.read()
args = Bunch(json.loads(args))
args.no_cuda = True
args.cuda = False
args.dropout= 0.0
if args.dataset == "traffic":
    args.output_dim = args.x_dim
else: #args.dataset == "human":
    args.output_dim = args.x_dim * 2

In [ ]:
print(args.dataset, args.model)

In [ ]:
print(args)

In [ ]:
dataDir = "/Users/danielzeiberg/Documents/Data/Human/Processed/INPUT_HORIZON_25_PREDICTION_HORIZON_50/"
dataDict = getDataLoaders(dataDir)

In [ ]:
def show2D(img, predPoints, targetPoints):
    ref = {"nJoints" : 16,
           "accIdxs" : [0, 1, 2, 3, 4, 5, 10, 11, 14, 15],
           "shuffleRef" : [[0, 5], [1, 4], [2, 3], 
                 [10, 15], [11, 14], [12, 13]],
           "edges" : [[0, 1], [1, 2], [2, 6], [6, 3], [3, 4], [4, 5], 
             [10, 11], [11, 12], [12, 8], [8, 13], [13, 14], [14, 15], 
             [6, 8], [8, 9]]
          }
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.33
    fontColor = (255,255,255)
    lineType = 2
    for points, c, of in zip([predPoints, targetPoints], [(255,0,0), (0,0,255)], [0,100]):
        points = ((points.reshape(ref["nJoints"], -1))).astype(np.int32)
        for e in ref["edges"]:
            cv2.line(img, (points[e[0], 0]+of, points[e[0], 1]), (points[e[1], 0]+of, points[e[1], 1]), (0,0,0), 2)
        for j in range(ref["nJoints"]):
            #cv2.putText(img, str(j), (points[j, 0], points[j, 1]), font, fontScale, fontColor, lineType)
            cv2.circle(img, (points[j, 0]+of, points[j, 1]), 5, c, -1)
    return img

In [ ]:
getlossobj = PostProcess(baseDir,args, dataDict=dataDict, chooseModel="rnn")
getlossobj.prep("rnn_full_model.pth", "train", -1)

In [ ]:
f = h5py.File(os.path.join(dataDir, "train.h5"), "r")

In [ ]:
inputTimes = np.arange(1, 25 * 5 + 1, 5)
targetTimes = np.arange(1 + 25*5, 1 + 75*5, 5)

In [ ]:
allTargets = []
allOutputs = []
allInputs = []
loss = nn.L1Loss()
for batchStop in range(64, 64 * 100, 64):
    print(batchStop-64, batchStop)
    inputT = torch.Tensor(f["input2d"][batchStop - 64:batchStop])
    allInputs.append(inputT)
    targetT = torch.Tensor(f["target2d"][batchStop - 64:batchStop])
    transInput, transTarget = dataDict["scaler"].transformBatchForEpoch((inputT, targetT))
    output = getlossobj.model(transInput, transTarget, 0)
    reconFirstOutput = dataDict["scaler"].inverse_transform(output)
    reconFirstTarget = dataDict["scaler"].inverse_transform(transTarget)
    print(loss(reconFirstOutput, reconFirstTarget))
    allOutputs.append(reconFirstOutput.cpu().detach().numpy())
    allTargets.append(reconFirstTarget.cpu().detach().numpy())
allOutputs = np.concatenate(allOutputs, axis=0)
allInputs = np.concatenate(allInputs, axis=0)
allTargets = np.concatenate(allTargets, axis=0)

In [ ]:
allInputs.shape

In [ ]:
allOutputs.shape

In [ ]:
allTargets.shape

In [ ]:
allData = np.concatenate((np.transpose(allInputs, (0,1,3,2)), allOutputs), axis=1)

In [ ]:
allTimes = np.concatenate((inputTimes, targetTimes))

In [ ]:
allTimes

In [ ]:
def convNum(num):
    if num < 10:
        return "0"+str(num)
    else:
        return str(num)

In [ ]:
convNum(22)

In [ ]:
allInputs.shape

In [ ]:
for _ in range(50):
    inst = np.random.choice(allOutputs.shape[0])
    whiteImage = cv2.imread("/Users/danielzeiberg/Downloads/white.png")

    def getImg(timeStepIdx):
        imgString = "/Users/danielzeiberg/Documents/Data/Human/images/"
        imgString += "s_{}_act_{}_subact_{}_ca_{}/s_{}_act_{}_subact_{}_ca_{}_{}{}.jpg".format(
                            convNum(f["subject"][inst]), convNum(f["action"][inst]), convNum(f["subaction"][inst]), convNum(f["camera"][inst]),\
                            convNum(f["subject"][inst]), convNum(f["action"][inst]), convNum(f["subaction"][inst]), convNum(f["camera"][inst]),\
                            "0" *( 6 - len(str(allTimes[timeStepIdx]))), allTimes[timeStepIdx])
        criterion = nn.L1Loss()
        whiteImage = cv2.imread("/Users/danielzeiberg/Downloads/white.png")
        return show2D(whiteImage, allOutputs[inst,timeStepIdx].T, allTargets[inst,timeStepIdx].T)

    def getBase(timeStepIdx):
        imgString = "/Users/danielzeiberg/Documents/Data/Human/images/"
        imgString += "s_{}_act_{}_subact_{}_ca_{}/s_{}_act_{}_subact_{}_ca_{}_{}{}.jpg".format(
                            convNum(f["subject"][inst]), convNum(f["action"][inst]), convNum(f["subaction"][inst]), convNum(f["camera"][inst]),\
                            convNum(f["subject"][inst]), convNum(f["action"][inst]), convNum(f["subaction"][inst]), convNum(f["camera"][inst]),\
                            "0" *( 6 - len(str(allTimes[timeStepIdx]))), allTimes[timeStepIdx])

        criterion = nn.L1Loss()
        whiteImage = cv2.imread("/Users/danielzeiberg/Downloads/white.png")
        return show2D(whiteImage, allInputs[inst,timeStepIdx], allInputs[inst,timeStepIdx])
    # images = [getBase(i) for i in range(25)] +[whiteImage]+ [getImg(i) for i in range(50)]
    images = [getImg(i) for i in range(50)]
    predNum= 0
    while os.path.isfile(baseDir+"preds_{}.gif".format(predNum)):
        predNum += 1
    imageio.mimsave(baseDir+'preds_{}.gif'.format(predNum), images, fps=1)


In [ ]:
np.unique(f["subject"][:6336]), np.unique(f["action"][:6336]), np.unique(f["subaction"][:6336]), np.unique(f["camera"][:6336])